# Merit Matrix Monte Carlo Analysis - Multi-Configuration

**Цель исследования:** Валидация точности метода агрегированных распределений для расчета бюджета повышений заработной платы через Monte Carlo симуляции.

**Ключевой вопрос:** При каком размере группы сотрудников метод становится надежным (success rate ≥ 80%)?

## Методология

1. **Данные:** 395,325 сотрудников из Москвы (обзор компенсаций, май 2025)
2. **Группировка:** Сегментация по Company × Function (типичная практика HR)
3. **Тестирование:** 120 конфигураций merit matrices (2-7 CR bins × 3-7 ratings × 4 типа распределений)
4. **Симуляция:** 50,000 Monte Carlo итераций на группу
5. **Метрика успеха:** % симуляций, попавших в диапазон ±5% от целевого бюджета

## Workflow

1. **Configuration Section** → Параметры анализа и конфигурации матриц
2. **Data Loading** → Загрузка и подготовка данных о компенсациях
3. **Analysis Execution** → Запуск Monte Carlo симуляций для всех конфигураций
4. **Results** → Анализ результатов и сохранение данных

### Библиотеки и их назначение

**Научные вычисления:**
- `numpy` — векторизованные операции, массивы для матриц
- `pandas` — обработка табличных данных, группировка по Company×Function
- `scipy.stats.dirichlet` — perturbation распределений рейтингов в Monte Carlo

**Оптимизация производительности:**
- `numba` — JIT-компиляция критичного Monte Carlo loop (ускорение ~100x)
- `prange` — параллелизация симуляций на всех CPU cores

**Утилиты:**
- `time`, `datetime` — измерение времени выполнения
- `pathlib.Path` — кроссплатформенная работа с файлами
- `warnings` — отключение технических предупреждений для читаемости

**Визуализация:**
- `matplotlib`, `seaborn` — создание heatmaps и графиков результатов

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet
from numba import jit, prange
import time
from datetime import datetime
import warnings
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

## 2. Data Loading and Preprocessing

### Источник данных

**Обзор компенсаций:** Реальные данные о зарплатах от компаний-участников  
**Референтная дата:** 01 мая 2025 года  
**Охват:** Все регионы России, множество функций и грейдов  

### Этапы подготовки данных

#### Шаг 1: Загрузка и переименование
- Загружаем исходный parquet файл с данными обзора
- Переименовываем столбцы на английский для консистентности кода

#### Шаг 2: Географическая фильтрация
- **Выбираем только Москву** для однородности рынка труда
- Обоснование: Москва содержит наибольшую выборку по всем функциям и грейдам

#### Шаг 3: Расчет Midpoint
- **Midpoint** = медиана базовой зарплаты внутри Company × Function × Grade
- Отражает "серединную точку" диапазона, на которую ориентируется компания
- Использование медианы: устойчивость к выбросам, стандарт в compensation management

#### Шаг 4: Расчет Compa Ratio (CR)
- **CR = Базовая зарплата / Midpoint**
- CR < 1.0 → сотрудник получает ниже медианы своего грейда
- CR = 1.0 → сотрудник на медиане
- CR > 1.0 → сотрудник получает выше медианы

#### Шаг 5: Агрегация
- Убираем Grade из финального датасета
- CR уже учитывает позицию относительно грейда
- Merit matrices применяются ко всей функции независимо от грейда

### Результат
- **Итоговый датасет:** Company | Function | BP | CR
- Готов для группировки по Company × Function и анализа

In [102]:
# Загрузка исходных данных обзора компенсаций
df_original = pd.read_parquet('03_Final_to_Code_corrected.parquet.gzip')

In [103]:
# Переименование столбцов для удобства работы
df_original = df_original.rename(columns={'Регион/область (заполняется автоматически)': 'Region',
                                         'Название компании (заполняется автоматически)': 'Company',
                                         'Название функции (заполняется автоматически)': 'Function',
                                         'Грейд / Уровень обзора': 'Grade',
                                         'Базовый оклад (BP)': 'BP'})

In [104]:
# Фильтрация: только Москва для консистентности анализа
df_original = df_original[df_original['Region'] == 'Москва'].copy()

# Отбор необходимых столбцов
df_original = df_original[['Company', 'Function', 'Grade', 'BP']]

In [105]:
# Расчет Midpoint: медиана BP внутри каждой Company × Function × Grade
# Midpoint отражает серединную точку диапазона зарплат для позиции
midpoints = df_original.groupby(['Company', 
                                 'Function', 
                                 'Grade'])['BP'].median().reset_index()

# Устанавливаем названия столбца
midpoints = midpoints.rename(columns={'BP': 'Midpoint'})

# Объединение данных с рассчитанными midpoints
dataset = pd.merge(df_original, midpoints, on=['Company', 
                                 'Function', 
                                 'Grade'], how='inner')

# Расчет Compa Ratio (CR) для каждого сотрудника
# CR = текущая зарплата / midpoint грейда
# CR показывает позицию сотрудника относительно рыночной медианы
dataset['CR'] = dataset['BP'] / dataset['Midpoint']

# Финальная агрегация: убираем Grade
# Merit matrices применяются ко всей функции независимо от грейда
# CR уже учитывает позицию сотрудника относительно его грейда
dataset = dataset[['Company', 'Function', 'BP', 'CR']]

In [96]:
# Сохраняем данные для фиксации выборки
dataset.to_parquet('Moscow.parquet.gzip', compression='gzip')

## 3. Configuration Parameters

### Параметры исследования

Эта секция определяет ключевые параметры для всего анализа. Изменение этих значений 
позволяет адаптировать исследование под различные сценарии.

#### Входные/выходные файлы
- `INPUT_FILE`: Подготовленный датасет с данными Moscow
- `OUTPUT_DIR`: Директория для сохранения результатов всех конфигураций

#### Параметры Monte Carlo симуляции
- `N_SIMULATIONS = 50,000`: Количество итераций на группу
  - Больше итераций = выше точность оценки, но дольше время выполнения
  - 50K обеспечивает стабильные результаты с приемлемым временем

- `MIN_SAMPLE_SIZE = 10`: Минимум сотрудников для включения группы в анализ
  - Группы меньше этого размера пропускаются (высокая вариативность)

- `BUDGET_TOLERANCE_LOWER/UPPER = 5%`: Диапазон допустимого отклонения
  - Symmetric: ±5% означает, что бюджет от 95% до 105% от target считается успешным
  - Success rate = % симуляций, попавших в этот диапазон

- `CONCENTRATION = 250`: Параметр Dirichlet распределения
  - Контролирует variance perturbation вокруг целевого распределения
  - Высокое значение (250) → малые отклонения (~±2-3%)
  - Имитирует: менеджеры в целом следуют политике, но есть небольшие отклонения

#### Тестируемые конфигурации
- `CR_BINS_RANGE = [2, 3, 4, 5, 6, 7]`: Количество CR bins в матрице
- `RATINGS_RANGE = [3, 4, 5, 6, 7]`: Количество рейтингов
- **Типы распределений = 4** (normal, skewed_high, skewed_low, uniform)

**Итого: 6 × 5 × 4 = 120 конфигураций для тестирования**

### Цель параметризации
Проверить **робастность** метода агрегированных распределений:
- Зависит ли точность от размера матрицы?
- Зависит ли точность от типа распределения рейтингов?
- Или главный фактор — размер группы сотрудников?

In [ ]:
# File paths
INPUT_FILE = 'Moscow.parquet.gzip'  #
OUTPUT_DIR = 'merit_analysis_results'

# Analysis parameters
N_SIMULATIONS = 50_000          # Monte Carlo iterations per group
MIN_SAMPLE_SIZE = 10            # Minimum employees per Company+Function
BUDGET_TOLERANCE_LOWER = 0.05   # 5% below target budget tolerance
BUDGET_TOLERANCE_UPPER = 0.05   # 5% above target budget tolerance
CONCENTRATION = 250             # Dirichlet concentration parameter

# Matrix and distribution ranges to test
CR_BINS_RANGE = range(2, 8)     # Test 2, 3, 4, 5, 6, 7 CR bins
RATINGS_RANGE = range(3, 8)     # Test 3, 4, 5, 6, 7 ratings

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)

print("="*100)
print("CONFIGURATION")
print("="*100)
print(f"  Input file: {INPUT_FILE}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  CR bins to test: {list(CR_BINS_RANGE)}")
print(f"  Ratings to test: {list(RATINGS_RANGE)}")
print(f"  Simulations per group: {N_SIMULATIONS:,}")
print(f"  Budget tolerance: -{BUDGET_TOLERANCE_LOWER*100:.0f}% / +{BUDGET_TOLERANCE_UPPER*100:.0f}%")

# Calculate total combinations
total_configs = len(list(CR_BINS_RANGE)) * len(list(RATINGS_RANGE)) * 4  # 4 distribution types
print(f"  Total configurations to test: {total_configs}")
print("="*100)



## 4. Rating Distributions Configuration

### Типы распределений рейтингов

Определяем 4 типа распределений для тестирования различных сценариев оценки 
эффективности сотрудников. Каждый тип отражает различную философию performance management.

#### 1. Normal (Нормальное распределение)
**Философия:** Bell curve — большинство сотрудников "средние", меньшинство в хвостах
- **Применение:** Наиболее распространенный подход в компаниях
- **Пример (5 рейтингов):** 10% | 15% | 50% | 15% | 10%
- **Характеристика:** Симметричное распределение вокруг среднего рейтинга

#### 2. Skewed High (Скошенное вправо)
**Философия:** "High performers" — позитивное смещение оценок
- **Применение:** Компании с сильной performance culture, tech стартапы
- **Пример (5 рейтингов):** 5% | 10% | 20% | 30% | 35%
- **Характеристика:** Больше сотрудников получают высокие оценки

#### 3. Skewed Low (Скошенное влево)
**Философия:** "Tough grading" — консервативный подход к оценкам
- **Применение:** Традиционные компании, строгие стандарты
- **Пример (5 рейтингов):** 35% | 30% | 20% | 10% | 5%
- **Характеристика:** Больше сотрудников получают низкие оценки

#### 4. Uniform (Равномерное распределение)
**Философия:** "All ratings equally likely" — отсутствие предпочтений
- **Применение:** Baseline для сравнения, теоретический сценарий
- **Пример (5 рейтингов):** 20% | 20% | 20% | 20% | 20%
- **Характеристика:** Все рейтинги одинаково вероятны

### Адаптация к количеству рейтингов
- Каждый тип распределения определен для 3, 4, 5, 6, 7 рейтингов
- Паттерн распределения сохраняется независимо от количества рейтингов
- Все распределения нормализованы (сумма = 100%)

### Использование в Monte Carlo
- Эти распределения служат **целевыми** (target)
- В каждой итерации распределение пертрубируется через Dirichlet(α)
- Имитирует реальность: менеджеры стремятся к target, но есть вариация

In [ ]:
DISTRIBUTIONS = {
    'normal': {
        3: {1: 10, 2: 80, 3: 10},
        4: {1: 10, 2: 30, 3: 50, 4: 10},
        5: {1: 10, 2: 15, 3: 50, 4: 15, 5: 10},
        6: {1: 5, 2: 10, 3: 25, 4: 35, 5: 20, 6: 5},
        7: {1: 5, 2: 10, 3: 15, 4: 40, 5: 15, 6: 10, 7: 5}
    },
    'skewed_high': {
        3: {1: 10, 2: 30, 3: 60},
        4: {1: 5, 2: 15, 3: 35, 4: 45},
        5: {1: 5, 2: 10, 3: 20, 4: 30, 5: 35},
        6: {1: 5, 2: 5, 3: 10, 4: 20, 5: 30, 6: 30},
        7: {1: 3, 2: 5, 3: 7, 4: 15, 5: 20, 6: 25, 7: 25}
    },
    'skewed_low': {
        3: {1: 60, 2: 30, 3: 10},
        4: {1: 45, 2: 35, 3: 15, 4: 5},
        5: {1: 35, 2: 30, 3: 20, 4: 10, 5: 5},
        6: {1: 30, 2: 30, 3: 20, 4: 10, 5: 5, 6: 5},
        7: {1: 25, 2: 25, 3: 20, 4: 15, 5: 7, 6: 5, 7: 3}
    },
    'uniform': {
        3: {1: 33.33, 2: 33.33, 3: 33.34},
        4: {1: 25, 2: 25, 3: 25, 4: 25},
        5: {1: 20, 2: 20, 3: 20, 4: 20, 5: 20},
        6: {1: 16.67, 2: 16.67, 3: 16.66, 4: 16.67, 5: 16.67, 6: 16.66},
        7: {1: 14.29, 2: 14.29, 3: 14.28, 4: 14.28, 5: 14.29, 6: 14.29, 7: 14.28}
    }
}

# Display distributions
print("="*100)
print("RATING DISTRIBUTIONS")
print("="*100)
for dist_name, dist_configs in DISTRIBUTIONS.items():
    print(f"\n{dist_name.upper().replace('_', ' ')}:")
    print("-" * 80)
    for n_ratings, dist in dist_configs.items():
        print(f"  {n_ratings} ratings: {dist} (sum={sum(dist.values()):.2f}%)")

## 5. Merit Matrix Generator Functions

### Функции генерации матриц повышений

Этот модуль автоматически создает merit matrices различных размеров, следуя 
ключевым принципам справедливости compensation management.

#### Функция `generate_merit_matrix(n_cr_bins, n_ratings)`

**Принципы генерации:**

1. **Монотонность по рейтингу (horizontal):**
   - Higher rating → Higher merit increase
   - В каждом CR bin повышение растет слева направо

2. **Монотонность по CR (vertical):**
   - Higher CR → Lower merit increase
   - В каждом рейтинге повышение уменьшается сверху вниз

3. **Взвешивание факторов:**
   - Rating weight = 70% (primary differentiator)
   - CR weight = 30% (secondary adjustment)
   - Обоснование: Performance важнее текущей позиции в диапазоне

4. **Диапазон повышений:**
   - Minimum = 1% (worst case: high CR, low rating)
   - Maximum = 20% (best case: low CR, high rating)
   - Реалистичные значения для типичных merit cycles

**Алгоритм:**
```
For each cell (cr_idx, rating_idx):
  1. Normalize positions to [0, 1]
  2. cr_factor = 1 - cr_position (inverse: low CR = high merit)
  3. rating_factor = rating_position (direct: high rating = high merit)
  4. combined = 0.7 × rating + 0.3 × cr
  5. merit = 1% + combined × (20% - 1%)
```

#### Функция `generate_cr_bins(n_bins)`

**Создание границ CR bins:**

Определяет edges для категоризации сотрудников по Compa Ratio.
Bins становятся более детальными с увеличением n_bins, с фокусом на зоне 0.8-1.2 
(где концентрируется большинство сотрудников).

**Примеры конфигураций:**
- **2 bins:** [0, 1.0, ∞) — простое деление: ниже/выше рынка
- **3 bins:** [0, 0.85, 1.15, ∞) — ниже / на рынке / выше рынка
- **5 bins:** [0, 0.80, 0.90, 1.10, 1.20, ∞) — детальная градация
- **7 bins:** [0, 0.75, 0.85, 0.92, 1.08, 1.15, 1.25, ∞) — максимальная детализация

**Логика boundaries:**
- Более узкие интервалы около 1.0 (где большинство данных)
- Более широкие интервалы в хвостах (меньше данных)
- Последний bin всегда открыт справа (∞) для outliers

### Модифицируемость
**ВАЖНО:** Если требуются другие проценты повышений, измените:
- `min_merit` и `max_merit` в `generate_merit_matrix()`
- Weights (0.7 / 0.3) для изменения относительной важности rating vs CR

In [ ]:
def generate_merit_matrix(n_cr_bins, n_ratings):
    """
    Generate merit matrix following the logic:
    - Top-left (low CR, low rating) = moderate merit
    - Top-right (low CR, high rating) = highest merit
    - Bottom-left (high CR, low rating) = lowest merit
    - Bottom-right (high CR, high rating) = moderate-high merit
    """
    min_merit = 1.0    # Bottom-left corner
    max_merit = 20.0   # Top-right corner
    
    matrix = np.zeros((n_cr_bins, n_ratings))
    
    for cr_idx in range(n_cr_bins):
        for rating_idx in range(n_ratings):
            cr_position = cr_idx / (n_cr_bins - 1) if n_cr_bins > 1 else 0
            rating_position = rating_idx / (n_ratings - 1) if n_ratings > 1 else 0
            
            # Rating is more important (70%) than CR position (30%)
            rating_factor = rating_position
            cr_factor = 1 - cr_position  # Inverse because lower CR = higher merit
            combined_factor = (rating_factor * 0.7 + cr_factor * 0.3)
            
            # Scale to merit range
            merit = min_merit + (max_merit - min_merit) * combined_factor
            matrix[cr_idx, rating_idx] = round(merit, 1)
    
    return matrix

def generate_cr_bins(n_bins):
    """Generate CR bin edges based on number of bins."""
    if n_bins == 2:
        return np.array([0.0, 1.0, np.inf], dtype=np.float32), ['[0.00, 1.00)', '[1.00, ∞)']
    elif n_bins == 3:
        return np.array([0.0, 0.85, 1.15, np.inf], dtype=np.float32), ['[0.00, 0.85)', '[0.85, 1.15)', '[1.15, ∞)']
    elif n_bins == 4:
        return np.array([0.0, 0.80, 0.95, 1.15, np.inf], dtype=np.float32), ['[0.00, 0.80)', '[0.80, 0.95)', '[0.95, 1.15)', '[1.15, ∞)']
    elif n_bins == 5:
        return np.array([0.0, 0.80, 0.90, 1.10, 1.20, np.inf], dtype=np.float32), ['[0.00, 0.80)', '[0.80, 0.90)', '[0.90, 1.10)', '[1.10, 1.20)', '[1.20, ∞)']
    elif n_bins == 6:
        return np.array([0.0, 0.75, 0.85, 0.95, 1.10, 1.25, np.inf], dtype=np.float32), ['[0.00, 0.75)', '[0.75, 0.85)', '[0.85, 0.95)', '[0.95, 1.10)', '[1.10, 1.25)', '[1.25, ∞)']
    elif n_bins == 7:
        return np.array([0.0, 0.75, 0.85, 0.92, 1.08, 1.15, 1.25, np.inf], dtype=np.float32), ['[0.00, 0.75)', '[0.75, 0.85)', '[0.85, 0.92)', '[0.92, 1.08)', '[1.08, 1.15)', '[1.15, 1.25)', '[1.25, ∞)']
    else:
        raise ValueError(f"Unsupported number of CR bins: {n_bins}")

print("✅ Merit matrix generator functions defined")

## 6. Preview Merit Matrices

### Визуализация всех тестируемых матриц

Эта секция отображает все 30 merit matrices (6 CR bins × 5 ratings), которые будут 
использоваться в анализе. Перед запуском полного анализа критически важно:

1. **Проверить корректность значений** — соответствуют ли проценты бизнес-логике?
2. **Верифицировать монотонность** — увеличивается ли merit слева-направо и сверху-вниз?
3. **Оценить диапазоны** — разумны ли min/max значения для вашего контекста?

### Формат отображения

**Для каждой матрицы показываются:**
- **Размер:** n CR Bins × m Ratings
- **CR Bin labels:** Диапазоны для каждого бина (например, [0.85, 1.15))
- **Merit Matrix (%):** Таблица процентов повышений
- **Statistics:** Min, Max, Mean значений в матрице

### Интерпретация матрицы

**Ориентация:**
- **Rows (Vertical):** CR Bins
  - Bin 0 (top) = Lowest CR (лучшая позиция — платим ниже рынка)
  - Bin N (bottom) = Highest CR (худшая позиция — платим выше рынка)
- **Columns (Horizontal):** Ratings
  - Rating 1 (left) = Lowest performance
  - Rating N (right) = Highest performance

**Углы матрицы:**
- **Top-Right** (low CR, high rating): Максимальное повышение (~20%)
- **Bottom-Left** (high CR, low rating): Минимальное повышение (~1%)
- **Top-Left / Bottom-Right**: Промежуточные значения

### Модификация
Если матрицы не соответствуют вашей компенсационной политике, 
вернитесь в секцию 5 и измените параметры `generate_merit_matrix()`.

In [ ]:
def display_all_matrices():
    """Display all merit matrices for review."""
    print("="*100)
    print("MERIT MATRICES PREVIEW")
    print("="*100)
    print("\nFormat: Rows = CR Bins (top = low CR, bottom = high CR)")
    print("        Columns = Ratings (left = low, right = high)")
    print("="*100)
    
    for n_cr in CR_BINS_RANGE:
        for n_ratings in RATINGS_RANGE:
            matrix = generate_merit_matrix(n_cr, n_ratings)
            bin_edges, bin_labels = generate_cr_bins(n_cr)
            
            print(f"\n{'-'*100}")
            print(f"MATRIX: {n_cr} CR Bins × {n_ratings} Ratings")
            print(f"{'-'*100}")
            
            # Show CR bins
            print("\nCR Bins:")
            for i, label in enumerate(bin_labels):
                print(f"  Bin {i}: {label}")
            
            # Show matrix
            print("\nMerit Matrix (%):")
            df = pd.DataFrame(
                matrix,
                index=[f"CR_Bin_{i}" for i in range(n_cr)],
                columns=[f"Rating_{i+1}" for i in range(n_ratings)]
            )
            print(df.to_string())
            
            print(f"\nStats: Min={matrix.min():.1f}%, Max={matrix.max():.1f}%, Mean={matrix.mean():.1f}%")

# Display all matrices
display_all_matrices()

## 7. Numba-Optimized Simulation Function

### Критичная функция производительности

Это **ядро Monte Carlo симуляции** — самая вычислительно-интенсивная часть анализа.
Оптимизация здесь критична, так как функция вызывается миллионы раз:
- 120 конфигураций × ~300 групп × 50,000 итераций = **~1.8 миллиарда вызовов**

### Оптимизации производительности

#### 1. Numba JIT компиляция
- `@jit(nopython=True)`: Компилирует Python в машинный код
- **Ускорение:** ~100x по сравнению с чистым Python
- `nopython=True`: Строгий режим (только NumPy операции, без Python objects)

#### 2. Параллелизация
- `parallel=True`: Автоматическая параллелизация на доступные CPU cores
- `prange()`: Parallel range — каждая итерация выполняется независимо
- **Ускорение:** Линейное с количеством cores (4 cores → ~4x speedup)

#### 3. Vectorization
- Все случайные числа pre-generated вне цикла
- `all_sampled_probs`: [n_simulations × n_ratings] — заранее
- `all_random_values`: [n_simulations × n_employees] — заранее
- Устраняет overhead генерации RNG внутри горячего цикла

#### 4. Быстрое присвоение рейтингов
- Используется inverse transform sampling через cumulative probabilities
- O(R) сложность вместо O(R log R) для каждого сотрудника
- R = количество рейтингов (3-7)

### Алгоритм симуляции

```
Для каждой из 50,000 симуляций (параллельно):
  1. Взять pre-sampled распределение рейтингов (Dirichlet perturbation)
  2. Построить cumulative probabilities для fast sampling
  3. Для каждого сотрудника:
     a. Присвоить рейтинг (inverse transform)
     b. Найти merit% в матрице [cr_bin, rating]
     c. Рассчитать merit amount = base_salary × merit%
     d. Накопить в total
  4. Рассчитать average merit% = total / total_base_salary
  5. Проверить: budget_lower ≤ average merit% ≤ budget_upper
```

### Производительность
- **Без оптимизаций:** ~300-500 секунд на группу из 100 сотрудников
- **С оптимизациями:** ~2-3 секунды на группу из 100 сотрудников
- **Speedup:** ~150x общее ускорение

### Технические детали
- `fastmath=True`: Разрешает compiler оптимизировать floating-point арифметику
- `cache=True`: Кэширует скомпилированную версию между запусками
- Все массивы используют `np.float32` для экономии памяти (важно при 50K итераций)

In [ ]:
@jit(nopython=True, parallel=True, fastmath=True, cache=True)
def _run_simulations_weighted(n_simulations, n_employees, base_salaries, cr_bins,
                              total_base_salary, all_sampled_probs, all_random_values,
                              merit_matrix, budget_lower_bound, budget_upper_bound,
                              average_merit_pcts, total_merit_amounts, within_budget):
    """Numba-optimized Monte Carlo simulation loop with parallelization."""
    n_ratings = merit_matrix.shape[1]
    
    for sim_idx in prange(n_simulations):
        sampled_probs = all_sampled_probs[sim_idx]
        
        # Build cumulative probabilities
        cum_probs = np.empty(n_ratings, dtype=np.float32)
        cum_probs[0] = sampled_probs[0]
        for i in range(1, n_ratings):
            cum_probs[i] = cum_probs[i-1] + sampled_probs[i]
        
        total_merit_amount = 0.0
        
        for emp_idx in range(n_employees):
            rand_val = all_random_values[sim_idx, emp_idx]
            
            # Assign rating
            rating_idx = 0
            for i in range(n_ratings):
                if rand_val <= cum_probs[i]:
                    rating_idx = i
                    break
            
            # Calculate merit
            cr_bin = cr_bins[emp_idx]
            merit_pct = merit_matrix[cr_bin, rating_idx]
            merit_amount = base_salaries[emp_idx] * (merit_pct / 100.0)
            total_merit_amount += merit_amount
        
        avg_merit_pct = (total_merit_amount / total_base_salary) * 100.0
        
        average_merit_pcts[sim_idx] = avg_merit_pct
        total_merit_amounts[sim_idx] = total_merit_amount
        within_budget[sim_idx] = (budget_lower_bound <= avg_merit_pct <= budget_upper_bound)

print("Numba simulation function compiled")

## 8. FlexibleMeritAnalyzer Class

### Центральный класс для анализа merit matrices

Этот класс инкапсулирует всю логику анализа, работая с матрицами любого размера.
Обеспечивает модульность и переиспользуемость кода для различных конфигураций.

### Архитектура класса

#### Инициализация `__init__()`
**Входы:**
- `merit_matrix`: NumPy array с процентами повышений
- `cr_bin_edges`: Границы CR bins (например, [0, 0.85, 1.15, ∞])
- `cr_bin_labels`: Текстовые метки для bins

**Сохраняет:**
- Конфигурацию матрицы (n_cr_bins, n_ratings)
- Все параметры для последующих вычислений

#### Ключевые методы

**1. `calculate_cr_distribution()`**
- Распределяет сотрудников по CR bins
- Вычисляет два типа распределений:
  - By count: процент сотрудников в каждом бине
  - By base pay: процент фонда оплаты труда в каждом бине (weighted)
- **Используется weighted distribution** для расчета бюджета (более точно)

**2. `calculate_heuristic_budget()`**
- Реализация **метода агрегированных распределений**
- Формула: `Budget = Σᵢ Σⱼ P(CR_i) × P(Rating_j) × Merit(i,j)`
- Предполагает независимость CR и рейтингов
- **Это метод, точность которого мы валидируем**

**3. `run_monte_carlo_for_group()`**
- Запускает Monte Carlo для одной группы (Company × Function)
- **Workflow:**
  1. Подготовка данных (assign CR bins, normalize probabilities)
  2. Pre-generation случайных чисел (Dirichlet samples + uniform RV)
  3. Вызов Numba-оптимизированной функции симуляции
  4. Вычисление метрик (success rate, percentiles, statistics)
- **Возвращает:** Dictionary с полными результатами (mean, median, std, percentiles, etc.)

**4. `process_dataframe()`**
- Обрабатывает весь датасет (все Company × Function группы)
- **Pipeline:**
  1. Валидация входных данных (columns, distribution normalization)
  2. Группировка по Company × Function
  3. Фильтрация малых групп (< min_sample_size)
  4. Для каждой группы:
     - Расчет CR distribution
     - Расчет heuristic budget
     - Запуск Monte Carlo
     - Сохранение результатов
  5. Формирование результирующего DataFrame
- **Возвращает:** Полная таблица результатов для всех групп

### Валидация и обработка ошибок
- Проверка наличия required columns (Company, Function, BP, CR)
- Валидация rating_distribution (keys = 1 to n_ratings, sum = 100%)
- Автоматический skip групп меньше min_sample_size
- Подробный verbose output для мониторинга прогресса

### Производительность
- Оптимизирован через NumPy vectorization где возможно
- Использует Numba для критичного simulation loop
- Прогресс-репорты каждые 5 групп для длительных runs
- Типичная скорость: ~200-300 групп в 10-15 минут (50K симуляций)

### Модульность
Класс полностью независим от:
- Размера матрицы (работает с любыми n_cr_bins × n_ratings)
- Типа распределения (принимает любое валидное распределение)
- Параметров симуляции (configurable через аргументы)

In [ ]:
class FlexibleMeritAnalyzer:
    """Merit analyzer that works with any matrix configuration."""
    
    def __init__(self, merit_matrix, cr_bin_edges, cr_bin_labels):
        self.merit_matrix = np.array(merit_matrix, dtype=np.float32)
        self.cr_bin_edges = np.array(cr_bin_edges, dtype=np.float32)
        self.cr_bin_labels = cr_bin_labels
        self.n_cr_bins = len(cr_bin_labels)
        self.n_ratings = merit_matrix.shape[1]
    
    def calculate_cr_distribution(self, cr_values, base_pays):
        """Calculate CR bin distribution weighted by base pay."""
        cr_bins = np.searchsorted(self.cr_bin_edges[1:-1], cr_values, side='right')
        
        cr_bin_counts = np.bincount(cr_bins, minlength=self.n_cr_bins)
        cr_bin_percentages = (cr_bin_counts / len(cr_bins)) * 100
        
        total_bp = np.sum(base_pays)
        cr_bin_bp = np.array([np.sum(base_pays[cr_bins == i]) for i in range(self.n_cr_bins)])
        cr_bin_bp_pct = (cr_bin_bp / total_bp) * 100
        
        return cr_bin_counts, cr_bin_percentages, cr_bin_bp, cr_bin_bp_pct
    
    def calculate_heuristic_budget(self, cr_distribution_pct, rating_distribution):
        """Calculate heuristic budget."""
        cr_dist = np.array(cr_distribution_pct, dtype=np.float32) / 100
        rating_dist = np.array([rating_distribution[i] for i in range(1, self.n_ratings + 1)], dtype=np.float32) / 100
        
        total_budget = 0.0
        for i in range(self.n_cr_bins):
            for j in range(self.n_ratings):
                cell_probability = cr_dist[i] * rating_dist[j]
                cell_increase = self.merit_matrix[i, j]
                total_budget += cell_probability * cell_increase
        
        return total_budget
    
    def run_monte_carlo_for_group(self, cr_values, base_pays, target_merit_pool_pct,
                                   rating_distribution, concentration,
                                   n_simulations=10000, budget_tolerance_lower=0.05,
                                   budget_tolerance_upper=0.0):
        """Run Monte Carlo simulation for a group."""
        n_employees = len(cr_values)
        total_base_salary = np.sum(base_pays)
        
        cr_bins = np.searchsorted(self.cr_bin_edges[1:-1], cr_values, side='right').astype(np.int32)
        
        target_probs = np.array([rating_distribution[i] for i in range(1, self.n_ratings + 1)], dtype=np.float32)
        
        budget_lower_bound = target_merit_pool_pct * (1 - budget_tolerance_lower)
        budget_upper_bound = target_merit_pool_pct * (1 + budget_tolerance_upper)
        
        average_merit_pcts = np.zeros(n_simulations, dtype=np.float32)
        total_merit_amounts = np.zeros(n_simulations, dtype=np.float32)
        within_budget = np.zeros(n_simulations, dtype=np.bool_)
        
        alpha = concentration * target_probs
        all_sampled_probs = dirichlet.rvs(alpha, size=n_simulations).astype(np.float32)
        all_random_values = np.random.random((n_simulations, n_employees)).astype(np.float32)
        
        _run_simulations_weighted(
            n_simulations=n_simulations,
            n_employees=n_employees,
            base_salaries=base_pays,
            cr_bins=cr_bins,
            total_base_salary=total_base_salary,
            all_sampled_probs=all_sampled_probs,
            all_random_values=all_random_values,
            merit_matrix=self.merit_matrix,
            budget_lower_bound=budget_lower_bound,
            budget_upper_bound=budget_upper_bound,
            average_merit_pcts=average_merit_pcts,
            total_merit_amounts=total_merit_amounts,
            within_budget=within_budget
        )
        
        success_rate = np.mean(within_budget) * 100
        
        return {
            'n_employees': n_employees,
            'total_base_salary': float(total_base_salary),
            'target_budget': target_merit_pool_pct,
            'budget_lower': budget_lower_bound,
            'budget_upper': budget_upper_bound,
            'mean': np.mean(average_merit_pcts),
            'median': np.median(average_merit_pcts),
            'std': np.std(average_merit_pcts),
            'p10': np.percentile(average_merit_pcts, 10),
            'p25': np.percentile(average_merit_pcts, 25),
            'p50': np.percentile(average_merit_pcts, 50),
            'p75': np.percentile(average_merit_pcts, 75),
            'p90': np.percentile(average_merit_pcts, 90),
            'min': np.min(average_merit_pcts),
            'max': np.max(average_merit_pcts),
            'p05': np.percentile(average_merit_pcts, 5),
            'p95': np.percentile(average_merit_pcts, 95),
            'iqr': np.percentile(average_merit_pcts, 75) - np.percentile(average_merit_pcts, 25),
            'mean_merit_amount': np.mean(total_merit_amounts),
            'median_merit_amount': np.median(total_merit_amounts),
            'std_merit_amount': np.std(total_merit_amounts),
            'success_rate': success_rate,
            'n_simulations': n_simulations,
            'n_within_budget': int(np.sum(within_budget)),
            'n_outside_budget': int(n_simulations - np.sum(within_budget))
        }
    
    def process_dataframe(self, df, rating_distribution, concentration=250,
                         n_simulations=10000, min_sample_size=25,
                         budget_tolerance_lower=0.05, budget_tolerance_upper=0.0,
                         verbose=True):
        """Process dataframe for all Company+Function groups."""
        required_cols = ['Company', 'Function', 'BP', 'CR']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        if not isinstance(rating_distribution, dict) or set(rating_distribution.keys()) != set(range(1, self.n_ratings + 1)):
            raise ValueError(f"rating_distribution must be a dict with keys 1-{self.n_ratings}")
        if abs(sum(rating_distribution.values()) - 100) > 0.01:
            raise ValueError(f"rating_distribution must sum to 100")
        
        results = []
        total_groups = len(df.groupby(['Company', 'Function']))
        processed_groups = 0
        skipped_groups = 0
        
        if verbose:
            print(f"  Processing {total_groups} Company+Function combinations...")
        
        start_time = time.time()
        
        for (company, function), group_df in df.groupby(['Company', 'Function']):
            n_employees = len(group_df)
            
            if n_employees < min_sample_size:
                skipped_groups += 1
                continue
            
            cr_values = group_df['CR'].values.astype(np.float32)
            base_pays = group_df['BP'].values.astype(np.float32)
            
            cr_bin_counts, cr_bin_pct_count, cr_bin_bp, cr_bin_pct_bp = \
                self.calculate_cr_distribution(cr_values, base_pays)
            
            heuristic_budget = self.calculate_heuristic_budget(cr_bin_pct_bp, rating_distribution)
            
            mc_results = self.run_monte_carlo_for_group(
                cr_values=cr_values,
                base_pays=base_pays,
                target_merit_pool_pct=heuristic_budget,
                rating_distribution=rating_distribution,
                concentration=concentration,
                n_simulations=n_simulations,
                budget_tolerance_lower=budget_tolerance_lower,
                budget_tolerance_upper=budget_tolerance_upper
            )
            
            result_row = {
                'company': company,
                'function': function,
                'n_employees': n_employees,
                'total_base_salary': mc_results['total_base_salary'],
                'heuristic_budget_pct': heuristic_budget,
                'target_budget_pct': mc_results['target_budget'],
                'budget_lower_pct': mc_results['budget_lower'],
                'budget_upper_pct': mc_results['budget_upper'],
                'mc_mean_pct': mc_results['mean'],
                'mc_median_pct': mc_results['median'],
                'mc_std_pct': mc_results['std'],
                'mc_p05_pct': mc_results['p05'],
                'mc_p10_pct': mc_results['p10'],
                'mc_p25_pct': mc_results['p25'],
                'mc_p50_pct': mc_results['p50'],
                'mc_p75_pct': mc_results['p75'],
                'mc_p90_pct': mc_results['p90'],
                'mc_p95_pct': mc_results['p95'],
                'mc_min_pct': mc_results['min'],
                'mc_max_pct': mc_results['max'],
                'mc_iqr_pct': mc_results['iqr'],
                'mc_mean_amount': mc_results['mean_merit_amount'],
                'mc_median_amount': mc_results['median_merit_amount'],
                'mc_std_amount': mc_results['std_merit_amount'],
                'success_rate_pct': mc_results['success_rate'],
                'n_within_budget': mc_results['n_within_budget'],
                'n_outside_budget': mc_results['n_outside_budget'],
                'n_simulations': mc_results['n_simulations'],
            }
            
            for i in range(self.n_cr_bins):
                result_row[f'cr_bin_{i}_count'] = int(cr_bin_counts[i])
                result_row[f'cr_bin_{i}_pct_count'] = cr_bin_pct_count[i]
                result_row[f'cr_bin_{i}_base_salary'] = float(cr_bin_bp[i])
                result_row[f'cr_bin_{i}_pct_bp'] = cr_bin_pct_bp[i]
            
            results.append(result_row)
            processed_groups += 1
        
        if verbose:
            elapsed_time = time.time() - start_time
            print(f"  ✓ Processed: {processed_groups} | Skipped: {skipped_groups} | Time: {elapsed_time:.1f}s")
        
        results_df = pd.DataFrame(results)
        if len(results_df) > 0:
            results_df = results_df.sort_values(['company', 'function']).reset_index(drop=True)
        
        return results_df

print("FlexibleMeritAnalyzer class defined")

## 9. Load Data

### Загрузка подготовленных данных

Загружаем датасет Moscow.parquet.gzip, созданный в секции 2.
Код поддерживает множественные форматы для гибкости:

**Поддерживаемые форматы:**
- `.parquet` / `.parquet.gzip` — рекомендуется (быстро, компактно)
- `.xlsx` / `.xls` — Excel файлы
- `.csv` — текстовые файлы

### Валидация данных

После загрузки проверяется:
- Количество строк и столбцов
- Количество сотрудников
- Общий фонд базовых зарплат
- Уникальные компании и функции
- Количество групп Company × Function (единица анализа)
- Статистика Compa Ratio (mean, median, range)

### Критичность качества данных

**Required columns:**
- `Company`: Идентификатор компании (для группировки)
- `Function`: Функциональная линия (для группировки)
- `BP`: Базовая зарплата (для расчета бюджета)
- `CR`: Compa Ratio (для классификации по bins)

**Проверка здоровья данных:**
- CR должен быть в разумном диапазоне (типично 0.6-1.6)
- BP должен быть положительным
- Нет missing values в критичных колонках

### Ожидаемый результат
После успешной загрузки должны увидеть:
- Loaded message с именем файла
- Shape информацию (rows × columns)
- Статистику по компаниям и функциям
- CR статистику для понимания распределения данных

In [ ]:
print("="*100)
print("LOADING DATA")
print("="*100)

try:
    if INPUT_FILE.endswith('.xlsx') or INPUT_FILE.endswith('.xls'):
        df = pd.read_excel(INPUT_FILE)
    elif INPUT_FILE.endswith('.csv'):
        df = pd.read_csv(INPUT_FILE)
    elif INPUT_FILE.endswith('.parquet.gzip') or INPUT_FILE.endswith('.parquet'):
        df = pd.read_parquet(INPUT_FILE)
    else:
        raise ValueError(f"Unsupported file format: {INPUT_FILE}")
    
    print(f"✅ Loaded: {INPUT_FILE}")
    print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"   Total employees: {len(df):,}")
    print(f"   Total base salary: ${df['BP'].sum():,.2f}")
    print(f"   Companies: {df['Company'].nunique()}")
    print(f"   Functions: {df['Function'].nunique()}")
    print(f"   Company+Function combinations: {df.groupby(['Company', 'Function']).ngroups}")
    
    print(f"\nCR Statistics:")
    print(f"   Mean: {df['CR'].mean():.3f}")
    print(f"   Median: {df['CR'].median():.3f}")
    print(f"   Min: {df['CR'].min():.3f}, Max: {df['CR'].max():.3f}")

except Exception as e:
    print(f"❌ ERROR loading data: {str(e)}")
    raise

## 10. Run Multi-Configuration Analysis

### Главный цикл исследования

**ВНИМАНИЕ: Длительная операция!**  
Для 120 конфигураций с 50,000 симуляций каждая ожидайте **2-4 часа** выполнения на стандартном laptop.

### Структура цикла

**Тройной вложенный цикл:**
```
For each n_cr_bins in [2, 3, 4, 5, 6, 7]:                    # 6 вариантов
  For each n_ratings in [3, 4, 5, 6, 7]:                    # 5 вариантов
    For each distribution in [normal, skewed_high, ...]:    # 4 варианта
      → Создать конфигурацию
      → Запустить анализ для всех групп
      → Сохранить результаты
```
**Итого:** 6 × 5 × 4 = **120 конфигураций**

### Workflow для каждой конфигурации

**1. Подготовка**
- Генерация merit matrix нужного размера
- Генерация CR bin edges
- Получение целевого rating distribution
- Создание уникального имени конфигурации (например, `CR5_R5_normal`)

**2. Инициализация**
- Создание экземпляра FlexibleMeritAnalyzer
- Конфигурирование параметров симуляции

**3. Выполнение анализа**
- Вызов `analyzer.process_dataframe()`:
  - Обработка всех Company × Function групп
  - 50,000 Monte Carlo итераций на группу
  - Фильтрация групп < MIN_SAMPLE_SIZE
- Измерение времени выполнения

**4. Сохранение результатов**
- **Формат:** Parquet (быстрый, компактный)
- **Naming:** `merit_analysis_{config_name}_{timestamp}.parquet`
- **Metadata:** Добавляются столбцы config_name, n_cr_bins, n_ratings, distribution_type
- **Location:** `merit_analysis_results/` директория

**5. Сводная статистика**
Для каждой конфигурации сохраняется:
- Количество обработанных групп
- Общее количество сотрудников
- Средний heuristic budget
- Средний Monte Carlo budget
- **Средний success rate** (ключевая метрика!)
- Время обработки

### Мониторинг прогресса

**Output в консоли:**
```
[1/120] CR2_R3_normal
────────────────────────────────────────────────────────────────
  Processing 287 Company+Function combinations...
  ✓ Processed: 243 | Skipped: 44 | Time: 124.3s
    Saved: merit_analysis_results/merit_analysis_CR2_R3_normal_20250518_143022.parquet
     Mean budget: 10.23%
     Mean success rate: 52.3%
     Time: 124.3s
```

### Обработка пропущенных групп
Группы пропускаются если `n_employees < MIN_SAMPLE_SIZE`:
- Причина: Высокая вариативность в малых выборках
- Типично: ~10-20% групп пропускаются (очень малые функции)
- Не критично: Фокус на статистически значимых группах

### Итоговая статистика
После завершения всех 120 конфигураций:
- Общее время выполнения
- Количество обработанных конфигураций
- Среднее время на конфигурацию
- Путь к сохраненным файлам

In [ ]:
print("\n" + "="*100)
print("RUNNING MULTI-CONFIGURATION ANALYSIS")
print("="*100)

overall_start = time.time()
config_counter = 0
all_summaries = []

for n_cr in CR_BINS_RANGE:
    for n_ratings in RATINGS_RANGE:
        for dist_name, dist_configs in DISTRIBUTIONS.items():
            config_counter += 1
            
            # Get configuration
            merit_matrix = generate_merit_matrix(n_cr, n_ratings)
            cr_bin_edges, cr_bin_labels = generate_cr_bins(n_cr)
            rating_distribution = dist_configs[n_ratings]
            
            config_name = f"CR{n_cr}_R{n_ratings}_{dist_name}"
            
            print(f"\n[{config_counter}/{total_configs}] {config_name}")
            print("-" * 100)
            
            # Initialize analyzer
            analyzer = FlexibleMeritAnalyzer(merit_matrix, cr_bin_edges, cr_bin_labels)
            
            # Run analysis
            config_start = time.time()
            
            results_df = analyzer.process_dataframe(
                df=df,
                rating_distribution=rating_distribution,
                concentration=CONCENTRATION,
                n_simulations=N_SIMULATIONS,
                min_sample_size=MIN_SAMPLE_SIZE,
                budget_tolerance_lower=BUDGET_TOLERANCE_LOWER,
                budget_tolerance_upper=BUDGET_TOLERANCE_UPPER,
                verbose=True
            )
            
            config_time = time.time() - config_start
            
            if len(results_df) > 0:
                # Add metadata
                results_df['config_name'] = config_name
                results_df['n_cr_bins'] = n_cr
                results_df['n_ratings'] = n_ratings
                results_df['distribution_type'] = dist_name
                
                # Save results
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                output_file = f"{OUTPUT_DIR}/merit_analysis_{config_name}_{timestamp}.parquet"
                results_df.to_parquet(output_file, index=False, compression='snappy')
                
                # Summary statistics
                summary = {
                    'config_name': config_name,
                    'n_cr_bins': n_cr,
                    'n_ratings': n_ratings,
                    'distribution_type': dist_name,
                    'n_groups': len(results_df),
                    'total_employees': results_df['n_employees'].sum(),
                    'total_base_salary': results_df['total_base_salary'].sum(),
                    'mean_heuristic_budget': results_df['heuristic_budget_pct'].mean(),
                    'mean_mc_budget': results_df['mc_mean_pct'].mean(),
                    'mean_success_rate': results_df['success_rate_pct'].mean(),
                    'processing_time_sec': config_time,
                    'output_file': output_file
                }
                all_summaries.append(summary)
                
                print(f"  ✅ Saved: {output_file}")
                print(f"     Mean budget: {summary['mean_mc_budget']:.2f}%")
                print(f"     Mean success rate: {summary['mean_success_rate']:.1f}%")
                print(f"     Time: {config_time:.1f}s")
            else:
                print(f"     No groups met minimum sample size")

overall_time = time.time() - overall_start

print("\n" + "="*100)
print("ANALYSIS COMPLETE")
print("="*100)
print(f"Total time: {overall_time/60:.1f} minutes")

## 11. Save and Display Summary

### Агрегация результатов всех конфигураций

После завершения анализа 120 конфигураций создается сводная таблица 
для сравнения performance различных настроек.

### Сохранение summary

**Файл:** `analysis_summary_{timestamp}.csv`

**Содержит для каждой конфигурации:**
- `config_name`: Уникальный идентификатор (например, CR5_R5_normal)
- `n_cr_bins`, `n_ratings`: Размеры матрицы
- `distribution_type`: Тип распределения рейтингов
- `n_groups`: Количество обработанных групп
- `mean_mc_budget`: Средний бюджет по Monte Carlo
- `mean_success_rate`: **Ключевая метрика** — средний success rate
- `processing_time_sec`: Время выполнения конфигурации

### Интерактивное отображение

**Jupyter display():** Показывает отфильтрованную версию summary для быстрого overview:
- Только основные столбцы для читаемости
- Отсортировано по config_name для группировки похожих конфигураций

### Использование summary

**Этот файл критичен для последующего анализа:**
1. Сравнение success rates между конфигурациями
2. Выявление паттернов: какие параметры влияют на точность?
3. Идентификация оптимальных конфигураций
4. Анализ времени выполнения (performance profiling)

### Структура для следующих шагов

Summary DataFrame сохраняется в переменной `results_summary` для:
- Секции 12 (Visualization) — создание heatmaps и графиков
- Дальнейшего статистического анализа
- Экспорта в презентации и отчеты

In [ ]:
if all_summaries:
    summary_df = pd.DataFrame(all_summaries)
    summary_file = f"{OUTPUT_DIR}/analysis_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    summary_df.to_csv(summary_file, index=False)
    
    print(f"✅ Summary saved: {summary_file}")
    print(f"\nTotal configurations processed: {len(all_summaries)}")
    print(f"Average time per configuration: {overall_time/len(all_summaries):.1f}s")
    
    print("\n" + "="*100)
    print("SUMMARY RESULTS")
    print("="*100)
    display(summary_df[['config_name', 'n_groups', 'mean_mc_budget', 'mean_success_rate', 'processing_time_sec']])
    
    # Store summary_df for visualization
    results_summary = summary_df
else:
    print("No results generated")

## 12. Visualization of Results

### Визуальный анализ результатов

Создание 4 ключевых визуализаций для понимания паттернов в данных.
Все графики сохраняются в высоком разрешении (300 DPI) для использования 
в презентациях и публикациях.

### График 1: Mean Monte Carlo Budget by Matrix Size (Heatmap)

**Что показывает:** Как средний бюджет зависит от размера матрицы
- **X-axis:** Количество рейтингов (3-7)
- **Y-axis:** Количество CR bins (2-7)
- **Color:** Средний бюджет (%) — агрегация по всем типам распределений

**Интерпретация:**
- Более теплые цвета (green) = выше бюджет
- Более холодные цвета (red) = ниже бюджет
- Ожидание: относительно стабильные значения (если размер матрицы не влияет)

### График 2: Mean Success Rate by Matrix Size (Heatmap)

**Что показывает:** Как точность метода зависит от размера матрицы
- **X-axis:** Количество рейтингов
- **Y-axis:** Количество CR bins
- **Color:** Средний success rate (%) — ключевая метрика исследования

**Интерпретация:**
- Более теплые цвета (green) = выше точность
- Более холодные цвета (red) = ниже точность
- **Ключевой вопрос:** Есть ли systematic pattern или значения стабильны?

### График 3: Mean Budget by Distribution Type (Bar Chart)

**Что показывает:** Как тип распределения рейтингов влияет на средний бюджет
- **Bars:** 4 типа распределений (normal, skewed_high, skewed_low, uniform)
- **Sorted:** От наименьшего к наибольшему для наглядности

**Интерпретация:**
- Skewed high → обычно выше бюджет (больше высоких рейтингов)
- Skewed low → обычно ниже бюджет (больше низких рейтингов)
- Разброс показывает sensitivity к типу распределения

### График 4: Mean Success Rate by Distribution Type (Bar Chart)

**Что показывает:** Как тип распределения влияет на точность метода
- **Bars:** 4 типа распределений
- **Sorted:** От наименьшего к наибольшему success rate

**Интерпретация:**
- Если разброс **малый** (2-5%) → метод robust к типу распределения
- Если разброс **большой** (>10%) → тип распределения — значимый фактор
- **Ключевой вопрос:** Влияет ли shape распределения на точность?

### Сохранение и использование

**Файл:** `analysis_summary_charts.png` (300 DPI, высокое качество)

**Применение:**
- Включение в README.md
- Использование в презентации защиты
- Публикация в статье
- Обсуждение на встречах с HR-командой

In [ ]:
if all_summaries:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Mean budget by configuration
    ax1 = axes[0, 0]
    summary_pivot = results_summary.pivot_table(
        values='mean_mc_budget', 
        index='n_cr_bins', 
        columns='n_ratings'
    )
    sns.heatmap(summary_pivot, annot=True, fmt='.2f', cmap='RdYlGn', ax=ax1)
    ax1.set_title('Mean Monte Carlo Budget (%) by Matrix Size')
    ax1.set_xlabel('Number of Ratings')
    ax1.set_ylabel('Number of CR Bins')
    
    # Plot 2: Success rate by configuration
    ax2 = axes[0, 1]
    success_pivot = results_summary.pivot_table(
        values='mean_success_rate', 
        index='n_cr_bins', 
        columns='n_ratings'
    )
    sns.heatmap(success_pivot, annot=True, fmt='.1f', cmap='RdYlGn', ax=ax2)
    ax2.set_title('Mean Success Rate (%) by Matrix Size')
    ax2.set_xlabel('Number of Ratings')
    ax2.set_ylabel('Number of CR Bins')
    
    # Plot 3: Budget by distribution type
    ax3 = axes[1, 0]
    dist_summary = results_summary.groupby('distribution_type')['mean_mc_budget'].mean().sort_values()
    dist_summary.plot(kind='barh', ax=ax3, color='steelblue')
    ax3.set_title('Mean Budget by Distribution Type')
    ax3.set_xlabel('Mean MC Budget (%)')
    
    # Plot 4: Success rate by distribution type
    ax4 = axes[1, 1]
    dist_success = results_summary.groupby('distribution_type')['mean_success_rate'].mean().sort_values()
    dist_success.plot(kind='barh', ax=ax4, color='coral')
    ax4.set_title('Mean Success Rate by Distribution Type')
    ax4.set_xlabel('Mean Success Rate (%)')
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/analysis_summary_charts.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Charts saved to: {OUTPUT_DIR}/analysis_summary_charts.png")

## 13. Quick Analysis Functions

### Утилиты для быстрого анализа результатов

Этот модуль предоставляет helper functions для интерактивного исследования 
сохраненных результатов без необходимости перезапуска всего анализа.

### Функция `analyze_specific_config(config_name)`

**Назначение:** Детальный анализ конкретной конфигурации

**Использование:**
```python
df = analyze_specific_config('CR5_R5_normal')
```

**Что делает:**
1. Ищет parquet файл для указанной конфигурации
2. Загружает последнюю версию (если несколько runs)
3. Выводит сводную статистику:
   - Количество проанализированных групп
   - Общее количество сотрудников
   - Средний heuristic budget
   - Средний Monte Carlo budget
   - Средний success rate
   - Распределение success rate (через describe())
4. Возвращает полный DataFrame для дальнейшего анализа

**Возвращает:** DataFrame с результатами для всех групп или None если не найдено

**Применение:**
- Глубокий dive в конкретную конфигурацию
- Анализ outliers (группы с необычно низким/высоким success rate)
- Проверка гипотез о конкретных настройках

### Функция `compare_distributions(n_cr, n_ratings)`

**Назначение:** Сравнение всех 4 типов распределений для фиксированного размера матрицы

**Использование:**
```python
comp_df = compare_distributions(5, 5)  # Сравнить для матрицы 5×5
```

**Что делает:**
1. Загружает результаты для всех 4 distributions (normal, skewed_high, skewed_low, uniform)
2. Вычисляет средние метрики для каждого типа
3. Выводит comparison таблицу
4. Возвращает DataFrame для дальнейшего анализа

**Возвращает:** DataFrame с 4 строками (по одной на distribution) или None

**Применение:**
- Понять, насколько sensitive метод к типу распределения
- Идентифицировать "worst case" и "best case" распределения
- Quantify разброс success rate между распределениями

### Примеры использования

**Пример 1: Анализ "проблемной" конфигурации**
```python
# Из summary видим, что CR7_R7_skewed_low имеет низкий success rate
df = analyze_specific_config('CR7_R7_skewed_low')

# Детальный анализ: какие группы особенно страдают?
low_success = df[df['success_rate_pct'] < 40]
print(low_success[['company', 'function', 'n_employees', 'success_rate_pct']])
```

**Пример 2: Сравнение эффекта распределений**
```python
# Для стандартной матрицы 5×5
comp = compare_distributions(5, 5)

# Вычислить разброс
spread = comp['mean_success_rate'].max() - comp['mean_success_rate'].min()
print(f"Success rate varies by {spread:.1f}% across distributions")
```

### Расширяемость

Эти функции служат шаблонами для создания дополнительных utility functions:
- `analyze_by_group_size()` — группировка по размеру групп
- `compare_matrix_sizes()` — сравнение разных размеров матриц
- `identify_outliers()` — поиск аномальных результатов

In [ ]:
def analyze_specific_config(config_name):
    """Load and analyze a specific configuration result."""
    matching_files = list(Path(OUTPUT_DIR).glob(f"merit_analysis_{config_name}_*.parquet"))
    if not matching_files:
        print(f"❌ No files found for config: {config_name}")
        return None
    
    latest_file = max(matching_files, key=lambda p: p.stat().st_mtime)
    df = pd.read_parquet(latest_file)
    
    print(f"Configuration: {config_name}")
    print(f"="*80)
    print(f"Groups analyzed: {len(df)}")
    print(f"Total employees: {df['n_employees'].sum():,}")
    print(f"Mean heuristic budget: {df['heuristic_budget_pct'].mean():.2f}%")
    print(f"Mean MC budget: {df['mc_mean_pct'].mean():.2f}%")
    print(f"Mean success rate: {df['success_rate_pct'].mean():.1f}%")
    
    # Show distribution
    print("\nSuccess Rate Distribution:")
    print(df['success_rate_pct'].describe())
    
    return df

def compare_distributions(n_cr, n_ratings):
    """Compare all distribution types for a specific matrix size."""
    results = []
    for dist_name in DISTRIBUTIONS.keys():
        config_name = f"CR{n_cr}_R{n_ratings}_{dist_name}"
        matching_files = list(Path(OUTPUT_DIR).glob(f"merit_analysis_{config_name}_*.parquet"))
        if matching_files:
            latest_file = max(matching_files, key=lambda p: p.stat().st_mtime)
            df = pd.read_parquet(latest_file)
            results.append({
                'distribution': dist_name,
                'mean_budget': df['mc_mean_pct'].mean(),
                'mean_success_rate': df['success_rate_pct'].mean()
            })
    
    if results:
        comp_df = pd.DataFrame(results)
        print(f"\nComparison for {n_cr}×{n_ratings} Matrix:")
        print("="*80)
        print(comp_df.to_string(index=False))
        return comp_df
    else:
        print(f"No results found for {n_cr}×{n_ratings} matrix")
        return None

print("✅ Analysis helper functions defined")
print("\nUsage:")
print("  - analyze_specific_config('CR5_R5_normal')")
print("  - compare_distributions(5, 5)")

### 14. Собираем все результаты экспериментов в единый файл для дальнейшего анализа

In [4]:
all_files = sorted(Path(OUTPUT_DIR).glob("merit_analysis_*.parquet"))
combined = pd.concat((pd.read_parquet(p) for p in all_files), ignore_index=True)

# If the same group/config was saved multiple times, keep the last (newest) row
combined['__ts'] = pd.to_datetime(combined['config_name'].str.extract(r'_(\d{8}_\d{6})$')[0],
                                  format="%Y%m%d_%H%M%S", errors='coerce')
combined = (combined
            .sort_values(['company','function','config_name','__ts'])
            .drop_duplicates(['company','function','config_name'], keep='last')
            .drop(columns='__ts'))

combined.to_parquet(f"ALL_results.parquet", index=False, compression='gzip')

df = pd.read_parquet('ALL_results.parquet')